<span style="float:left;">Licence CC BY-SA</span><span style="float:right;">Fabrice Zaoui - Cedric Goeury&nbsp;</span><br/>
___

This tutorial is intended for people who want an example showing how to optimize a Telemac 2D case with the deterministic algorithm based on the SciPy package.

# Import Optimizer from TelApy

In [1]:
from telapy.tools import newop

The module 'newop' is located in the '$HOMETEL/scripts/python3/telapy/tools' directory. If an error occurs while attempting to import, check the value of the environment variable PYTHONPATH.


'newop' uses the SciPy optimizer named 'minimizer' based on the quasi-Newton deterministic algorithm L-BFGS-B.

In [2]:
help(newop)

Help on package telapy.tools.newop in telapy.tools:

NAME
    telapy.tools.newop - NEWOP : a Python (v3) Newton Optimizer

DESCRIPTION
    Based on the SciPy Minimizer function
    
    Auteur : Fabrice Zaoui (EDF R&D LNHE)
    
    Copyright EDF 2017-2018

PACKAGE CONTENTS
    newop
    numval
    validate

CLASSES
    builtins.object
        telapy.tools.newop.newop.Newop
    
    class Newop(builtins.object)
     |  Newop(d_x=1e-06, maxfun=2000, verbose=True)
     |  
     |  The base class for the SciPy Opimization
     |  
     |  Methods defined here:
     |  
     |  __init__(self, d_x=1e-06, maxfun=2000, verbose=True)
     |      Initialize some algorithmic parameters to default values
     |      :return: a new object from Newop
     |  
     |  initialize(self, func, nvar, bounds, vdx=None)
     |      Description of the minimization problem
     |      :param 'f': the name of the python function where the cost function
     |          is implemented (type: str)
     |      :

# The Telemac2d test case

A problem is created with the 'estimation' example located in '$HOMETEL/examples/telemac2d/estimation'.

The test case 'estimation' is dedicated to the automatic calibration problem. This Telemac-2D model is composed of 551 triangular mesh elements. On the upstream of the model, an imposed flow type boundary condition is used. On downstream, a water depth is applied.

The script 'study_t2d.py' has been written to manage this case with the Telemac API (initialization of the case, run of the case, get some important values on the physical variables).

In [3]:
from telapy.tools.study_t2d import StudyTelemac2D
import numpy as np
import os

# Goal

The calibration aims at minimizing the error between observations and Telemac computations on the water depths. Six observations are available on the middle of the domain. A norm is used to evaluate the error as follows:

In [4]:
def estimation(CHESTR):
    YObs = [5.000000000000000000e-01, 7.517873224063527093e-01, \
            7.517873219825667030e-01, 7.517873219442824384e-01, \
            7.517873221409325790e-01, 7.517873218929342904e-01]
    YObs = np.asmatrix(np.ravel(YObs)).T # six observations
    Fx = study.h_x(CHESTR[0]) # Telemac computation with new friction coefficient
    Res = np.linalg.norm(YObs -Fx) # The norm evaluation
    return np.array(Res)

# Reading the case

In [5]:
# Changing of directory : 'examples' directory of the Telemac sources
HOMETEL = os.environ.get('HOMETEL')
os.chdir(HOMETEL + '/examples/telemac2d/estimation')

In [6]:
#  Telemac 2d files
studyFiles={'t2d.f'   :'user_fortran',\
            't2d.cas' :'t2d_estimation_basic.cas',\
            'f2d.slf' :'f2d_estimation.slf',\
            't2d.geo' :'geo_estimation.slf' }

In [7]:
# Observation times
obs_time = [0.0, 2000.0, 4000.0, 6000.0, 8000.0, 10000.0]
# A polygon is defined to get the observation node
poly = [(246.114, 57.3554), (261.13, 57.0189), (261.802, 45.018), (245.666, 45.3545)]

# Initialization

In [8]:
# Class Instantiation 
study = StudyTelemac2D(studyFiles, obs_time, poly)

# Run the optimization

In [9]:
# number of variables
nvar = 1
# lower and upper bounds
vbounds = np.zeros((nvar, 2))
for i in range(nvar):
    vbounds[i, 0] = 20.
    vbounds[i, 1] = 60.
# initial guess
x0 = np.array([[50.]])
# instantiation of the optimizer class
mypb = newop.Newop()
# initialize the optimizer with : goal function, number of variables and associated bounds
error = mypb.initialize(estimation, nvar, vbounds)
# launch optimization with initial guess and a maximum number of processors for parallelism
val = mypb.optimize(x0, nproc=3)

  ~> Checking keyword/rubrique coherence

  ~> Checking keyword/rubrique coherence

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            1     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.92486D-01    |proj g|=  9.16087D-03


  ~> Checking keyword/rubrique coherence

  ~> Checking keyword/rubrique coherence

...

  ~> Checking keyword/rubrique coherence

  ~> Checking keyword/rubrique coherence

  ~> Checking keyword/rubrique coherence

  ~> Checking keyword/rubrique coherence


At iterate    7    f=  7.08385D-09    |proj g|=  2.96972D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    1      7     48      7     3     0   2.970D-03   7.084D-09
  F =   7.0838457607930183E-009

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


In [10]:
# Print the optimal solution for the friction coefficient
print(val[1])

[34.99999956]


In [11]:
# Print the corresponding error (2-norm)
print(val[0])

7.083845760793018e-09
